Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f, encoding = 'latin1')
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [13]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.175139
Minibatch accuracy: 6.2%
Validation accuracy: 14.3%
Minibatch loss at step 50: 1.707598
Minibatch accuracy: 25.0%
Validation accuracy: 45.9%
Minibatch loss at step 100: 0.959412
Minibatch accuracy: 68.8%
Validation accuracy: 70.8%
Minibatch loss at step 150: 0.477518
Minibatch accuracy: 81.2%
Validation accuracy: 74.2%
Minibatch loss at step 200: 0.926611
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 250: 1.242916
Minibatch accuracy: 62.5%
Validation accuracy: 77.6%
Minibatch loss at step 300: 0.445035
Minibatch accuracy: 87.5%
Validation accuracy: 78.8%
Minibatch loss at step 350: 0.543425
Minibatch accuracy: 93.8%
Validation accuracy: 77.3%
Minibatch loss at step 400: 0.350341
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 450: 0.906244
Minibatch accuracy: 75.0%
Validation accuracy: 79.1%
Minibatch loss at step 500: 0.757281
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Mi

Validation accuracy: 86.9%
Minibatch loss at step 4550: 0.315603
Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Minibatch loss at step 4600: 0.549210
Minibatch accuracy: 87.5%
Validation accuracy: 87.0%
Minibatch loss at step 4650: 0.813503
Minibatch accuracy: 81.2%
Validation accuracy: 86.5%
Minibatch loss at step 4700: 0.397114
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 4750: 0.823794
Minibatch accuracy: 68.8%
Validation accuracy: 86.7%
Minibatch loss at step 4800: 0.405073
Minibatch accuracy: 87.5%
Validation accuracy: 87.0%
Minibatch loss at step 4850: 0.365415
Minibatch accuracy: 93.8%
Validation accuracy: 87.1%
Minibatch loss at step 4900: 0.222141
Minibatch accuracy: 93.8%
Validation accuracy: 86.8%
Minibatch loss at step 4950: 0.287020
Minibatch accuracy: 87.5%
Validation accuracy: 87.0%
Minibatch loss at step 5000: 1.057139
Minibatch accuracy: 68.8%
Validation accuracy: 86.7%
Minibatch loss at step 5050: 0.183937
Minibatch accuracy: 100.0

Validation accuracy: 88.5%
Minibatch loss at step 9050: 0.346798
Minibatch accuracy: 87.5%
Validation accuracy: 88.1%
Minibatch loss at step 9100: 0.344653
Minibatch accuracy: 87.5%
Validation accuracy: 87.9%
Minibatch loss at step 9150: 0.723246
Minibatch accuracy: 75.0%
Validation accuracy: 87.9%
Minibatch loss at step 9200: 0.247540
Minibatch accuracy: 87.5%
Validation accuracy: 88.1%
Minibatch loss at step 9250: 0.562588
Minibatch accuracy: 81.2%
Validation accuracy: 88.4%
Minibatch loss at step 9300: 0.887882
Minibatch accuracy: 81.2%
Validation accuracy: 88.2%
Minibatch loss at step 9350: 0.204825
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 9400: 0.312319
Minibatch accuracy: 87.5%
Validation accuracy: 88.4%
Minibatch loss at step 9450: 0.484000
Minibatch accuracy: 81.2%
Validation accuracy: 88.4%
Minibatch loss at step 9500: 0.092911
Minibatch accuracy: 93.8%
Validation accuracy: 88.6%
Minibatch loss at step 9550: 0.234489
Minibatch accuracy: 87.5%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    
  # Add Max Pooling operation with stride 2 and a kernel size 2 
  def max_pool_2d(data, k):
        return tf.nn.max_pool(data, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
        
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pooling = max_pool_2d(hidden, 2)
    
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pooling = max_pool_2d(hidden, 2)
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [15]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.728968
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.709544
Minibatch accuracy: 43.8%
Validation accuracy: 53.5%
Minibatch loss at step 100: 0.998012
Minibatch accuracy: 68.8%
Validation accuracy: 66.7%
Minibatch loss at step 150: 0.461012
Minibatch accuracy: 87.5%
Validation accuracy: 74.2%
Minibatch loss at step 200: 0.849149
Minibatch accuracy: 81.2%
Validation accuracy: 76.8%
Minibatch loss at step 250: 1.160487
Minibatch accuracy: 68.8%
Validation accuracy: 76.8%
Minibatch loss at step 300: 0.342577
Minibatch accuracy: 93.8%
Validation accuracy: 78.8%
Minibatch loss at step 350: 0.571423
Minibatch accuracy: 93.8%
Validation accuracy: 77.8%
Minibatch loss at step 400: 0.303869
Minibatch accuracy: 93.8%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.900179
Minibatch accuracy: 81.2%
Validation accuracy: 77.9%
Minibatch loss at step 500: 0.758178
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Mi

Validation accuracy: 87.0%
Minibatch loss at step 4550: 0.243122
Minibatch accuracy: 100.0%
Validation accuracy: 87.2%
Minibatch loss at step 4600: 0.703347
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 4650: 0.715224
Minibatch accuracy: 81.2%
Validation accuracy: 86.4%
Minibatch loss at step 4700: 0.403116
Minibatch accuracy: 87.5%
Validation accuracy: 86.9%
Minibatch loss at step 4750: 0.824003
Minibatch accuracy: 68.8%
Validation accuracy: 86.6%
Minibatch loss at step 4800: 0.280698
Minibatch accuracy: 87.5%
Validation accuracy: 87.0%
Minibatch loss at step 4850: 0.224517
Minibatch accuracy: 93.8%
Validation accuracy: 86.9%
Minibatch loss at step 4900: 0.141684
Minibatch accuracy: 93.8%
Validation accuracy: 86.9%
Minibatch loss at step 4950: 0.294576
Minibatch accuracy: 93.8%
Validation accuracy: 87.1%
Minibatch loss at step 5000: 1.062678
Minibatch accuracy: 68.8%
Validation accuracy: 86.7%
Minibatch loss at step 5050: 0.298371
Minibatch accuracy: 87.5

Validation accuracy: 88.4%
Minibatch loss at step 9050: 0.272832
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 9100: 0.257801
Minibatch accuracy: 93.8%
Validation accuracy: 88.3%
Minibatch loss at step 9150: 0.843876
Minibatch accuracy: 68.8%
Validation accuracy: 86.9%
Minibatch loss at step 9200: 0.240178
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 9250: 0.618805
Minibatch accuracy: 75.0%
Validation accuracy: 88.0%
Minibatch loss at step 9300: 0.804669
Minibatch accuracy: 81.2%
Validation accuracy: 88.2%
Minibatch loss at step 9350: 0.172999
Minibatch accuracy: 93.8%
Validation accuracy: 88.0%
Minibatch loss at step 9400: 0.240584
Minibatch accuracy: 93.8%
Validation accuracy: 88.6%
Minibatch loss at step 9450: 0.610521
Minibatch accuracy: 75.0%
Validation accuracy: 88.2%
Minibatch loss at step 9500: 0.141718
Minibatch accuracy: 93.8%
Validation accuracy: 88.2%
Minibatch loss at step 9550: 0.360758
Minibatch accuracy: 87.5%

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [16]:
# will be updated soon ...